# Workflow
## 1. Define testbed
## 2. Define Scenario
## 3. Run Scenario on Testbed

# 1. Define Testbed

In [1]:
import sys

sys.path.append('../../')

from testbed import *

def get_connection(ip='', username='', password=''):
    con = connection('_ssh')
    con.add_attribute('_ip',ip)
    con.add_attribute('_username',username)
    con.add_attribute('_password',password)
    return con

def get_testbed():

    #mininet_server
    mininet_server = device(id='device_mininet',type='_mininet_server')
    mininet_server.add_os(osHeliot('_linux'))
    mininet_server.add_connection(get_connection('172.17.15.21','xx', 'xx'))

    #airsim_server
    airsim_server = device(id='device_airsim',type='_airsim_server')
    airsim_server.add_os(osHeliot('_windows'))
    airsim_server.add_connection(get_connection('172.17.49.168','xx', 'xx'))

    #Tx2
    nvidia_jetson_tx2 = device(id='device_tx2',type='_nvidia_jetson_tx2')
    nvidia_jetson_tx2.add_os(osHeliot('_linux'))
    nvidia_jetson_tx2.add_connection(get_connection('172.17.49.60','xx', 'xx'))

    #gvk
    google_vision_kit = device(id='device_gvk',type='_google_vision_kit')
    google_vision_kit.add_os(osHeliot('_linux'))
    google_vision_kit.add_connection(get_connection('172.17.49.65','xx', 'xx'))
    
    #smartphone
    android_smartphone = device(id='device_phone',type='_smartphone')
    android_smartphone.add_os(osHeliot('_android'))
    ## Require a special kind of REST API connection.
    android_smartphone.add_connection(connection('_rest'))

    my_testbed = testbed(name='my_demo_testbed')
    # Takes any user defined name as input, can be left empty

    # Adding devices
    my_testbed.add_device(mininet_server)
    #my_testbed.add_device(airsim_server)
    #my_testbed.add_device(nvidia_jetson_tx2)
    my_testbed.add_device(google_vision_kit)
    #my_testbed.add_device(android_smartphone)

    return my_testbed

In [2]:
my_testbed = get_testbed()

Adding device  device_mininet  to testbed
Adding device  device_gvk  to testbed


## Validating the testbed

In [3]:
# 1) Test the reachability of devices
# 2) Download the Heliot repo from github on devices

# 3) Starts the Heliot slaves on all the devices. 
## (Provides the RPC functionality to run Tasks)

# In Andriod and other non-linux/window based devices, 
# assumption is these are setup by the user. Eg: using Android client of Heliot

my_testbed.validate()

2019-02-03 16:54:44,874 - testbed - INFO - Validating the devices in Testbed
2019-02-03 16:54:46,910 - testbed - INFO - device_mininet is connected
2019-02-03 16:54:47,357 - testbed - INFO - heliot_runtime directory created
2019-02-03 16:54:48,985 - testbed - INFO - Heliot repo downloaded from Github
2019-02-03 16:54:49,032 - testbed - INFO - heliot initalization done for device_mininet
2019-02-03 16:54:51,071 - testbed - INFO - device_gvk is connected
2019-02-03 16:54:51,620 - testbed - INFO - heliot_runtime directory created
2019-02-03 16:54:54,600 - testbed - INFO - Heliot repo downloaded from Github
2019-02-03 16:54:54,655 - testbed - INFO - heliot initalization done for device_gvk
2019-02-03 16:54:54,656 - testbed - INFO - Testbed validated


# 2. Defining the Scenario

## Nodes

In [4]:
from scenario import *
myScenario = scenario(name='NESL_Simple_Scenario')

virtual_drone_node = airsimSensor(type='_drone', id = 'drone')
virtual_drone_node.add_attribute('_controller','file_path')
myScenario.add_airsimSensor(virtual_drone_node)

tx2_node = node(type='_nvidia_jetson_tx2', id = 'Tx2')
myScenario.add_node(tx2_node)

gvt_node = node(type='_google_vision_kit', id = 'gvk')
myScenario.add_node(gvt_node)

virtual_switch = infranode(type='_switch', id='vs')
myScenario.add_infranode(virtual_switch)

Adding airsimSensor  drone  to scenario
Adding node  Tx2  to scenario
Adding node  gvk  to scenario
Adding infranode  vs  to scenario


## Links between Nodes

In [5]:
link_tx2_vswitch= mininetLink(name='link_tx2_vswitch', id_1='Tx2',id_2='vs')
link_tx2_vswitch.add_attribute('_latency', '2') #latency in milli seconds

link_gvk_vswitch= mininetLink(name='link_gvk_vswitch',id_1='gvk',id_2='vs')
link_gvk_vswitch.add_attribute('_latency', '2') #latency in milli seconds

link_drone_vswitch= mininetLink(name='link_drone_vswitch', id_1='drone',id_2='vs')
link_drone_vswitch.add_attribute('_latency', '2') #latency in milli seconds

myScenario.add_mininetLink(link_tx2_vswitch)
myScenario.add_mininetLink(link_gvk_vswitch)
myScenario.add_mininetLink(link_drone_vswitch)

Adding mininetLink  link_tx2_vswitch  to scenario
Adding mininetLink  link_gvk_vswitch  to scenario
Adding mininetLink  link_drone_vswitch  to scenario


## Tasks and Dataflow between Tasks

In [6]:
task_gvt_camera = taskHeliot(_type='_compute',_taskid='task_gvt_camera',_filename='Code_to_run_task',_nodeid='image_sensor_1')
task_airsim_camera = taskHeliot(_type='_virtualSensor',_taskid='task_airsim_camera',_filename='Code_to_run_task',_nodeid='Surveillance_drone')
task_object_inference = taskHeliot(_type='_compute',_taskid='task_object_inference',_filename='Code_to_run_task',_nodeid='Tx2_image_classifier')

gvt_image_data = task_gvt_camera.get_output("gvt_image_data") 
drone_image_data = task_airsim_camera.get_output("drone_image_data")
task_object_inference.set_input([gvt_image_data,drone_image_data])  #It can take list of output objects

gvt_image_data : is set as output for task:  task_gvt_camera
drone_image_data : is set as output for task:  task_airsim_camera
gvt_image_data : is set as input for task: task_object_inference
drone_image_data : is set as input for task: task_object_inference


In [7]:
myScenario.add_task(task_gvt_camera)
myScenario.add_task(task_airsim_camera)
myScenario.add_task(task_object_inference)

Adding task: task_gvt_camera
Adding task: task_airsim_camera
Adding task: task_object_inference


In [8]:
## Node mapping and ## Task mapping are implicit here based on 
## Node_types and node_ids in the task definition

# 3. Run Scenario on Testbed

## Setting up the network topology as per Scenario

In [9]:
myScenario.start_network()

Adding infranodes to the network
Adding nodes to the network
Adding airSim sensors to the network


*** Ping: testing ping reachability
Tx20 -> 

Starting the network using Mininet


gvk0 drone0 
gvk0 -> Tx20 drone0 
drone0 -> Tx20 gvk0 
*** Results: 0% dropped (6/6 received)


## Starting the Scenario Tasks

In [10]:
myScenario.start_tasks(my_testbed)

Starting Task: task_gvt_camera on node: image_sensor_1
Starting Task: task_airsim_camera on node: Surveillance_drone
Starting Task: task_object_inference on node: Tx2_image_classifier
